读取数据集：收集了100个用户和其相关物品的数据集


In [1]:
import pandas as pd
order_data = pd.read_csv('data/GoodsOrder.csv', header=0, encoding='gbk')
# 转换数据格式
order_data['Goods'] = order_data['Goods'].apply(lambda x: "," + x)
order_data = order_data.groupby('id').sum().reset_index()
order_data['Goods'] = order_data['Goods'].apply(lambda x: [x[1:]])
order_data_list = list(order_data['Goods'])
# print(order_data_list)
# 分割商品名为每一个元素
data_translation = []
for i in order_data_list:
    # 列表中元素为1个字符串，所以用0索引
    p = i[0].split(",")
    data_translation.append(p)
print(len(data_translation))
data = {
    'ID': [],
    'Basket': []

}
for i in range(len(data_translation)):
    data['ID'].append(i)
    data['Basket'].append(data_translation[i])
data = pd.DataFrame(data)
print(data)

100
    ID                                             Basket
0    0                          [柑橘类水果, 人造黄油, 即食汤, 半成品面包]
1    1                                     [咖啡, 热带水果, 酸奶]
2    2                                             [全脂牛奶]
3    3                              [奶油乳酪, 肉泥, 仁果类水果, 酸奶]
4    4                              [炼乳, 长面包, 其他蔬菜, 全脂牛奶]
..  ..                                                ...
95  95                                         [葡萄, 相片/膜]
96  96  [黑面包, 罐装啤酒, 水果/蔬菜汁, 粉红葡萄酒, 面包卷, 香肠, 购物袋, 切片奶酪,...
97  97  [黄油, 鸡, 柑橘类水果, 一般清洁剂, 汉堡肉, 油, 其他蔬菜, 意大利面, 根茎类蔬...
98  98                                    [黑面包, 全脂牛奶, 酸奶]
99  99                                      [柑橘类水果, 热带水果]

[100 rows x 2 columns]


In [2]:
# 将 Basket 列取出来单独处理，然后再将处理好的数据拼接回去
print(" ID列 ".center(100, '='))
data_id = data.drop('Basket', 1)
print(data_id)

print(" Basket列 ".center(100, '='))
basket = data.Basket
print(basket)

=============================================== ID列 ================================================
    ID
0    0
1    1
2    2
3    3
4    4
..  ..
95  95
96  96
97  97
98  98
99  99

[100 rows x 1 columns]
============================================= Basket列 ==============================================
0                             [柑橘类水果, 人造黄油, 即食汤, 半成品面包]
1                                        [咖啡, 热带水果, 酸奶]
2                                                [全脂牛奶]
3                                 [奶油乳酪, 肉泥, 仁果类水果, 酸奶]
4                                 [炼乳, 长面包, 其他蔬菜, 全脂牛奶]
                            ...                        
95                                           [葡萄, 相片/膜]
96    [黑面包, 罐装啤酒, 水果/蔬菜汁, 粉红葡萄酒, 面包卷, 香肠, 购物袋, 切片奶酪,...
97    [黄油, 鸡, 柑橘类水果, 一般清洁剂, 汉堡肉, 油, 其他蔬菜, 意大利面, 根茎类蔬...
98                                      [黑面包, 全脂牛奶, 酸奶]
99                                        [柑橘类水果, 热带水果]
Name: Basket, Length: 100, dtype: object


D:\wlxuan\Anaconda3\envs\Datamining\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
print(" 将列表转化为字符串的Basket列 ".center(100, '='))
basket = data.Basket.str.join(',')
print(basket)

print(" 根据Basket列数据转化为数值型 ".center(100, '='))
basket = basket.str.get_dummies(',')
print(basket)
print(basket.columns.tolist())
col = basket.columns.tolist()
print(" 将数值型数据拼接回原数据 ".center(100, '='))
data = data_id.join(basket)
print(data)

======================================== 将列表转化为字符串的Basket列 =========================================
0                            柑橘类水果,人造黄油,即食汤,半成品面包
1                                      咖啡,热带水果,酸奶
2                                            全脂牛奶
3                                奶油乳酪,肉泥,仁果类水果,酸奶
4                                炼乳,长面包,其他蔬菜,全脂牛奶
                         ...                     
95                                        葡萄,相片/膜
96      黑面包,罐装啤酒,水果/蔬菜汁,粉红葡萄酒,面包卷,香肠,购物袋,切片奶酪,威化饼
97    黄油,鸡,柑橘类水果,一般清洁剂,汉堡肉,油,其他蔬菜,意大利面,根茎类蔬菜,热带水果
98                                    黑面包,全脂牛奶,酸奶
99                                     柑橘类水果,热带水果
Name: Basket, Length: 100, dtype: object
======================================== 根据Basket列数据转化为数值型 =========================================
    一般清洁剂  一般饮料  人造黄油  仁果类水果  全脂牛奶  其他蔬菜  其他饮料  冰淇淋  冷冻土豆制品  冷冻热狗肠  ...  面包卷  \
0       0     0     1      0     0     0     0    0       0      0  ...    0   
1       0     0     0      0     0     0     0 

In [4]:
from mlxtend.frequent_patterns import apriori, association_rules
# 用标准数据继续关联规则分析的步骤
# 利用mlxtend提供的apriori算法函数得到频繁项集
frequent_item_sets = apriori(data[col], min_support=0.02,
                             use_colnames=True)
print(frequent_item_sets)
# 计算规则，并设置提升度阈值为 1 （返回的是各个指标的数值，可以按照按兴趣的指标排序观察，但具体解释还得参考实际数据的含义）
rules = association_rules(frequent_item_sets, metric='lift', min_threshold=1)
print(rules)

     support                  itemsets
0       0.06                   (一般清洁剂)
1       0.02                    (一般饮料)
2       0.04                    (人造黄油)
3       0.02                   (仁果类水果)
4       0.25                    (全脂牛奶)
..       ...                       ...
260     0.02            (购物袋, 香肠, 面包卷)
261     0.02      (餐巾, 瓶装水, 面包卷, 卫生用品)
262     0.02    (热带水果, 威化饼, 糖果, 根茎类蔬菜)
263     0.02  (购物袋, 水果/蔬菜汁, 罐装啤酒, 黑面包)
264     0.02      (购物袋, 香肠, 面包卷, 罐装啤酒)

[265 rows x 2 columns]
     antecedents       consequents  antecedent support  consequent support  \
0        (一般清洁剂)            (其他蔬菜)                0.06                0.17   
1         (其他蔬菜)           (一般清洁剂)                0.17                0.06   
2        (根茎类蔬菜)           (一般清洁剂)                0.09                0.06   
3        (一般清洁剂)           (根茎类蔬菜)                0.06                0.09   
4         (热带水果)           (一般清洁剂)                0.12                0.06   
..           ...               ...      

D:\wlxuan\Anaconda3\envs\Datamining\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:114: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  DeprecationWarning,
